In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
df = pd.read_csv('./hospital_data.csv')

In [3]:
df.head()

,Region,District,FacilityName,Type,Town,Ownership,Latitude,Longitude
0,Ashanti,Offinso North,A.M.E Zion Clinic,Clinic,Afrancho,CHAG,7.40801,-1.96317
1,Ashanti,Bekwai Municipal,Abenkyiman Clinic,Clinic,Anwiankwanta,Private,6.46312,-1.58592
2,Ashanti,Adansi North,Aboabo Health Centre,Health Centre,Aboabo No 2,Government,6.22393,-1.34982
3,Ashanti,Afigya-Kwabre,Aboabogya Health Centre,Health Centre,Aboabogya,Government,6.84177,-1.61098
4,Ashanti,Kwabre,Aboaso Health Centre,Health Centre,Aboaso,Government,6.84177,-1.61098


In [11]:
df.shape

(3756, 8)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3756 entries, 0 to 3755
Data columns (total 8 columns):
Region          3756 non-null object
District        3756 non-null object
FacilityName    3756 non-null object
Type            3756 non-null object
Town            3603 non-null object
Ownership       3756 non-null object
Latitude        3732 non-null float64
Longitude       3732 non-null float64
dtypes: float64(2), object(6)
memory usage: 234.9+ KB


In [5]:
df.describe()

,Latitude,Longitude
count,3732.000000,3732.000000
mean,6.955678,-1.074580
std,1.679332,1.021352
min,4.796180,-3.235880
25%,5.687642,-1.809792
50%,6.468705,-1.058117
75%,7.430555,-0.215870
max,11.146670,1.191440


In [8]:
df.Type.unique()

array(['Clinic', 'Health Centre', 'Maternity Home', 'CHPS', 'Hospital',
       'District Hospital', 'Centre', 'RCH', 'Training Institution',
       'Municipal Health Directorate', 'Teaching Hospital',
       'Regional Hospital', 'CPHS', 'District Health Directorate',
       'Regional Health Directorate', 'Polyclinic', 'Municipal Hospital',
       'Municipal  Health Directorate', 'Metropolitan Hospital',
       'Metropolitan Health Directorate', 'DHD', 'Psychiatric Hospital',
       'Research Institution', 'Others', 'clinic'], dtype=object)

In [9]:
df.Type.value_counts()

Clinic                             1171
Health Centre                       786
CHPS                                652
Maternity Home                      369
Hospital                            277
RCH                                 152
District Health Directorate          99
District Hospital                    82
Training Institution                 74
Others                               31
Polyclinic                           16
Regional Hospital                     9
Regional Health Directorate           9
Municipal Health Directorate          7
Municipal Hospital                    4
Teaching Hospital                     3
Psychiatric Hospital                  3
Centre                                2
Research Institution                  2
clinic                                2
Metropolitan Health Directorate       2
Metropolitan Hospital                 1
CPHS                                  1
DHD                                   1
Municipal  Health Directorate         1


In [12]:
df.Ownership.value_counts()

Government          2210
Private             1179
CHAG                 257
Quasi-Government      91
government             6
Islamic                4
Maternity Home         2
NGO                    2
private                2
Clinic                 1
Muslim                 1
Mission                1
Name: Ownership, dtype: int64

In [13]:
df.Ownership.unique()

array(['CHAG', 'Private', 'Government', 'Quasi-Government', 'Islamic',
       'Muslim', 'Maternity Home', 'Clinic', 'private', 'NGO',
       'government', 'Mission'], dtype=object)

In [22]:
# Missing towns
missing_towns = df[df.Town.isnull()]
missing_towns.shape

(153, 8)

In [20]:
# missing coordinates
missing_coordinates = df[df.Latitude.isnull()|df.Latitude.isnull()]
missing_coordinates.shape

(24, 8)

## Assessment

## Cleaning

## Visualization